In [1]:
!pip install tensorflow

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
import os
# from google.colab import drive
# USE_GOOGLE_DRIVE = True
# if USE_GOOGLE_DRIVE:
#     drive.mount('/content/drive')
#     PATH_PREFIX = 'drive/My Drive/ABSA'
# else:
#     PATH_PREFIX = '..'

train_data=os.path.join("/content/Laptop_Train_v2.xml")
test_data=os.path.join("/content/Laptops_Test_Gold.xml")

# Aspect Extraction - using ML models

In [5]:
import pandas as pd


# convert into bio format
class BIO:

    def convert_into_bio(self, text, predicted_aspect_terms_list, common_words_list):
        text_list = text.split()
        # print(text_list)
        df = pd.DataFrame({'text': text_list, 'BIO': 'O'})

        bio_dict = self.bio_mapper(common_words_list=common_words_list, prediction=predicted_aspect_terms_list)
        bio_filtered_dict = self.filter_predicted_words(bio_dict)

        for item in bio_filtered_dict:
            final_list = self.matcher(text, item)
            if final_list[1] - final_list[0] == 0:
                df.at[final_list[0], 'BIO'] = 'B'
            else:
                df.at[final_list[0], 'BIO'] = 'B'
                for index in range(final_list[0] + 1, final_list[1] + 1):
                        df.at[index, 'BIO'] = 'I'

        return df

    def bio_mapper(self, common_words_list, prediction):
        bio_dict = dict()

        for i in range(len(common_words_list)):
            bio_dict[common_words_list[i]] = prediction[i]

        return bio_dict

    def matcher(self, text, word):
        split_word = word.split()
        split_text = text.split(word)
        start=0
        end=0
        if len(split_text) > 1:
            start = len(split_text[0].split())
            end = start + len(split_word) - 1
        return start, end

    def filter_predicted_words(self, _dict):

        final_list = []

        for key, value in _dict.items():
            if value == 1:
                final_list.append(key)

        return final_list;


def main():

    text = "The battery life is really good and its size is reasonable"
    x = BIO()
    prd = [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

    common_words = ['OS',
                    'Vista',
                    'Windows',
                    'Windows 7',
                    'applications',
                    'battery',
                    'battery life',
                    'carry',
                    'charge',
                    'cost',
                    'design',
                    'display',
                    'extended warranty',
                    'features',
                    'games',
                    'gaming',
                    'graphics',
                    'hard drive',
                    'keyboard',
                    'keys',
                    'look',
                    'memory',
                    'motherboard',
                    'mouse',
                    'operating system',
                    'performance',
                    'power',
                    'power supply',
                    'price',
                    'processor',
                    'program',
                    'programs',
                    'quality',
                    'runs',
                    'screen',
                    'service',
                    'shipping',
                    'size',
                    'software',
                    'speakers',
                    'speed',
                    'system',
                    'use',
                    'value',
                    'warranty',
                    'warrenty',
                    'weight',
                    'windows',
                    'work',
                    'works']

    print(x.convert_into_bio(text, prd, common_words_list=common_words))

if __name__ == '__main__':
  main()

          text BIO
0          The   O
1      battery   B
2         life   I
3           is   O
4       really   O
5         good   O
6          and   O
7          its   O
8         size   B
9           is   O
10  reasonable   O


In [6]:
import xml.etree.ElementTree as ET
import pandas as pd


pd.set_option('display.max_colwidth', -1)

class Xml2DataFrame:

    @staticmethod
    def parse_root(root):
        """Return pandas dataframe from given xml data"""
        data_list = []
        xml_data = dict()

        target_words = []
        for child in root:
            xml_data['id'] = child.attrib.get('id')
            text = child.find('text').text.lower()
            xml_data['text'] = text
            for aspectTerms in child.iter('aspectTerms'):
                aspectInfos = []
                for asp_term in aspectTerms:
                    aspectInfos.append(asp_term.attrib)
                    # print(asp_term.attrib)
                xml_data['aspect_info'] = aspectInfos
            data_list.append(xml_data)
            xml_data = {}

        return data_list

    def process_data(self, path):
        root = ET.parse(path).getroot()
        """ Initiate the root XML, parse it, and return a dataframe"""
        structure_data = self.parse_root(root)
        df = pd.DataFrame([[k.get('id'), k.get('text'), k.get('aspect_info')] for k in iter(structure_data)],
                          columns=['id', 'text', 'aspect_info'])
        return df


if __name__ == '__main__':
    xml2df = Xml2DataFrame()
    xml_dataframe = xml2df.process_data(test_data)


<ipython-input-6-c56b53af3594>:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [7]:
import nltk
from nltk.corpus import stopwords
from string import punctuation


class POSTagger:

    @staticmethod
    def pos_tagger(text):
        text = nltk.word_tokenize(text)
        stopwords_en = stopwords.words('english')
        stopwords_en_withpunct = set(stopwords_en).union(set(punctuation))
        text = [word for word in text if word not in stopwords_en_withpunct]
        tagged_pos_list = nltk.pos_tag(text)
        return tagged_pos_list

    def filter_pos_tag(self, tagged_text):
        final_text_list = []
        matching_tag = ['NN','NNS','NNP','NNPS','RB','RBR','RBS','JJ','JJR','JJS','VB','VBD','VBG','VBN','VBP','VBZ']

        for word, tag in tagged_text:
            final_text = []
            if tag in matching_tag:
                final_text.append(word)
                final_text_list.append((' '.join(final_text)))
        return self.combine(final_text_list)

    @staticmethod
    def combine(filtered_tags):
        filtered_string = ' '.join(filtered_tags)
        return filtered_string


if __name__ == '__main__':
    x = POSTagger()
    k = x.pos_tagger("The battery life is really good and its size is reasonable")
    y = x.filter_pos_tag(k)

In [15]:
import pandas as pd
import joblib
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from scipy.sparse import hstack
import warnings



In [16]:
# xml parser
def get_xml_data(path):
    xml2df = Xml2DataFrame()
    xml_dataframe = xml2df.process_data(path)
    return xml_dataframe

#get_xml_data(train_data)

In [17]:
train_dataframe = get_xml_data(train_data)
# print(train_dataframe.head())
train_text_list = train_dataframe['text']
train_aspects_list = list(train_dataframe['aspect_info'])
print(train_text_list.head())
print("=============================")
print(train_aspects_list[:5])

0    i charge it at night and skip taking the cord with me because of the good battery life.                                                                                            
1    i bought a hp pavilion dv4-1222nr laptop and have had so many problems with the computer.                                                                                          
2    the tech guy then said the service center does not do 1-to-1 exchange and i have to direct my concern to the "sales" team, which is the retail shop which i bought my netbook from.
3    i investigated netbooks and saw the toshiba nb305-n410bl.                                                                                                                          
4    the other day i had a presentation to do for a seminar at a large conference in town- lots of people, little time to prep and have to set up a computer to a projector, etc.       
Name: text, dtype: object
[[{'term': 'cord', 'polarity': 'neutral', 'from':

In [18]:

# POSTag function wrapper
def pos_tag(review):
    tagged_text_list = []
    pos_tagger = POSTagger()
    for text in review:
        tagged_text_list.append(pos_tagger.pos_tagger(text=text))
    return tagged_text_list


# POSTag filtering of noun,adjective,verb,adverb
def filter_tag(tagged_reviews):
    filtered_list = []
    pos_tagger = POSTagger()
    for tagged_review in tagged_reviews:
        filtered_list.append(pos_tagger.filter_pos_tag(tagged_review))
    return filtered_list

In [19]:
# tag POS
tagged_text_list_train = pos_tag(train_text_list)
joblib.dump(tagged_text_list_train, 'tagged_text_list_train.pkl')
tagged_text_list_train = joblib.load('tagged_text_list_train.pkl')
# print(tagged_text_list_train[:5])

In [20]:
# train list after filtering
final_train_text_list = filter_tag(tagged_text_list_train)

print(final_train_text_list[:5])

['charge night skip taking cord good battery life', 'bought hp pavilion dv4-1222nr laptop many problems computer', 'tech guy said service center 1-to-1 exchange direct concern sales team retail shop bought netbook', 'investigated netbooks saw toshiba nb305-n410bl', 'day presentation seminar large conference town- lots people little time prep set computer projector etc']


In [21]:
#Selecting only 20 most common aspect.
def get_most_common_aspect(aspect_list):
    import nltk
    aspect_terms = []

    aspect_list = list(aspect_list.aspect_info)

    for inner_list in aspect_list:
        if inner_list is not None:
            for _dict in inner_list:
                # for key in _dict:
                aspect_terms.append(_dict.get('term'))

    most_common_aspect = [k for k, v in nltk.FreqDist(aspect_terms).most_common(1000)]
    return most_common_aspect

In [22]:
# generate data frame
def get_data_frame(text_list, train_aspects_list, most_common_aspect):
    data = {'Text': text_list}
    df = pd.DataFrame(data)
    for inner_list in train_aspects_list:
        if inner_list is not None:
            for _dict in inner_list:
                # for key in _dict:
                if _dict.get('term') in most_common_aspect:
                    df.loc[train_aspects_list.index(inner_list), _dict.get('term')] = _dict.get('polarity')
    return df

In [23]:

# generate data frame for aspect extraction task
def get_aspect_data_frame(df, most_common_aspect):
    for common_aspect in most_common_aspect:
        df[common_aspect] = df[common_aspect].replace(['positive','negative','neutral','conflict'],[1,1,1,1])
    df = df.fillna(0)
    return df

In [24]:

most_common_aspect = get_most_common_aspect(train_dataframe)
print(most_common_aspect)

['screen', 'price', 'use', 'battery life', 'battery', 'keyboard', 'programs', 'software', 'features', 'warranty', 'hard drive', 'quality', 'size', 'performance', 'speed', 'Windows', 'memory', 'graphics', 'applications', 'motherboard', 'runs', 'Vista', 'gaming', 'system', 'program', 'works', 'charge', 'design', 'display', 'power supply', 'Windows 7', 'service', 'operating system', 'windows', 'warrenty', 'value', 'keys', 'mouse', 'OS', 'speakers', 'extended warranty', 'shipping', 'games', 'cost', 'work', 'processor', 'look', 'hardware', 'power', 'weight', 'carry', 'fan', 'Keyboard', 'touchpad', 'trackpad', 'feature', 'boot up', 'webcam', 'internet', 'sound', 'touch pad', 'iWork', 'screen size', 'Windows 7 Starter', 'tech support', 'Screen', 'drivers', 'navigate', 'hinge', 'mousepad', 'iTunes', 'cd drive', 'space', 'iPhoto', 'RAM', 'edges', 'buttons', 'portability', 'service center', 'mother board', 'web browsing', 'iMovie', 'monitor', 'iLife', 'set up', 'HD', 'DVD burner', 'USB ports', '

In [28]:
#get data frame
import warnings
warnings.filterwarnings("ignore")
df_train = get_data_frame(final_train_text_list,train_aspects_list, most_common_aspect)

In [29]:
# get expect term df
df_train_aspect = get_aspect_data_frame(df_train, most_common_aspect)
df_train_aspect.head()

,Text,cord,battery life,service center,"""sales"" team",tech guy,quality,GUI,applications,use,...,Price,Value,WiFi,update programs,MS applications,Internet tabs,noises,bottom of the computer,repair technician,repair
0,charge night skip taking cord good battery life,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,bought hp pavilion dv4-1222nr laptop many problems computer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,tech guy said service center 1-to-1 exchange direct concern sales team retail shop bought netbook,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,investigated netbooks saw toshiba nb305-n410bl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,day presentation seminar large conference town- lots people little time prep set computer projector etc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
df_train_aspect = df_train_aspect.reindex(sorted(df_train_aspect.columns), axis=1)
df_train_aspect.head()

,"""sales"" team",1 GB ram,1-year-warranty,10-key,12 cell battery,13 inch,15 inch,"15""",16GB RAM support,17 ince screen,...,word,word editing,word processer,word processing,word processing program,word processor,work,working,works,wt
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
# Similar for test list
test_dataframe = get_xml_data(test_data)
test_text_list = train_dataframe['text']
test_aspects_list = list(train_dataframe['aspect_info'])

In [32]:
tagged_text_list_test = pos_tag(test_text_list)
joblib.dump(tagged_text_list_test, 'tagged_text_list_test.pkl')
tagged_text_list_test=joblib.load('tagged_text_list_test.pkl')

In [33]:
final_test_text_list = filter_tag(tagged_text_list_test)

In [34]:
df_test = get_data_frame(final_test_text_list,test_aspects_list, most_common_aspect)
df_test_aspect = get_aspect_data_frame(df_test, most_common_aspect)
df_test_aspect = df_test_aspect.reindex(sorted(df_test_aspect.columns), axis=1)

In [35]:
# Sort the data frame according to aspect's name and separate data(X) and target(y)
# df_train_aspect = df_train_aspect.sample(frac=1).reset_index(drop=True) # For randomization
X_train= df_train_aspect.Text
y_train = df_train_aspect.drop('Text', 1)
print(y_train[:5])


# df_test_aspect = df_test_aspect.sample(frac=1).reset_index(drop=True) # For randomization
X_test = df_test_aspect.Text
y_test = df_test_aspect.drop('Text', 1)
final_most_common_aspect = list(y_train)


   "sales" team  1 GB ram  1-year-warranty  10-key  12 cell battery  13 inch  \
0  0.0           0.0       0.0              0.0     0.0              0.0       
1  0.0           0.0       0.0              0.0     0.0              0.0       
2  1.0           0.0       0.0              0.0     0.0              0.0       
3  0.0           0.0       0.0              0.0     0.0              0.0       
4  0.0           0.0       0.0              0.0     0.0              0.0       

   15 inch  15"  16GB RAM support  17 ince screen  ...  word  word editing  \
0  0.0      0.0  0.0               0.0             ...  0.0   0.0            
1  0.0      0.0  0.0               0.0             ...  0.0   0.0            
2  0.0      0.0  0.0               0.0             ...  0.0   0.0            
3  0.0      0.0  0.0               0.0             ...  0.0   0.0            
4  0.0      0.0  0.0               0.0             ...  0.0   0.0            

   word processer  word processing  word processin

In [36]:
# Change y_train to numpy array
import numpy as np
y_train = np.asarray(y_train, dtype=np.int64)
y_test = np.asarray(y_test, dtype=np.int64)
print(y_train[:5])

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [37]:
# Generate word vecotors using CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# from nltk import word_tokenize
# from nltk.stem import WordNetLemmatizer
vect = CountVectorizer(max_df=1.0, stop_words='english')
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

In [38]:
# Create various models. These are multi-label models.
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
nb_classif = OneVsRestClassifier(MultinomialNB()).fit(X_train_dtm, y_train)
C = 1.0
# SVregularization parameter
svc = OneVsRestClassifier(svm.SVC(kernel='linear', C=C)).fit(X_train_dtm, y_train)
lin_svc = OneVsRestClassifier(svm.LinearSVC(C=C)).fit(X_train_dtm, y_train)
sgd = OneVsRestClassifier(SGDClassifier(max_iter=1000)).fit(X_train_dtm,y_train)



In [39]:
clf_sent =tree.DecisionTreeClassifier().fit(X_train_dtm,y_train)

In [40]:
# Predict the test data using classifiers
y_pred_class = nb_classif.predict(X_test_dtm)
y_pred_class_svc = svc.predict(X_test_dtm)
y_pred_class_lin_svc = lin_svc.predict(X_test_dtm)
y_pred_class_sgd = sgd.predict(X_test_dtm)
y_pred_sent=clf_sent.predict(X_test_dtm)

In [41]:
!pip3 install --upgrade tensorflow

In [42]:
from keras.preprocessing.text import Tokenizer

vocab_size = 3644 # We set a maximum size for the vocabulary
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X_train)
aspect_tokenized = pd.DataFrame(tokenizer.texts_to_matrix(X_train))
#aspect_tokenized

In [43]:
# Following code to test metrics of all aspect extraction classifiers
from sklearn import metrics

In [44]:

print('NB-classifier:',metrics.accuracy_score(y_test,y_pred_class))
print('SVM',metrics.accuracy_score(y_test,y_pred_class_svc))
#print('Linear SVM:',metrics.accuracy_score(y_test,y_pred_class_lin_svc))
#print('SGD classifier',metrics.accuracy_score(y_test,y_pred_class_sgd))
#print('RF classifier',metrics.accuracy_score(y_test,y_pred_class_rf))
print('tree classifier',metrics.accuracy_score(y_test,y_pred_sent))

NB-classifier: 0.5822660098522168
SVM 0.9894909688013136
tree classifier 0.9973727422003285


In [45]:
print(metrics.precision_score(y_test,y_pred_class,average='micro'))
print(metrics.precision_score(y_test,y_pred_class_svc,average='micro'))
print(metrics.precision_score(y_test,y_pred_class_lin_svc,average='micro'))
print(metrics.precision_score(y_test,y_pred_class_sgd,average='micro'))

0.9232558139534883
0.9968138370505234
0.9968340117593849
0.9954358740301232


In [46]:
# stored trained model
joblib.dump(sgd, 'trained_sgd.pkl')

['trained_sgd.pkl']

In [47]:


def BIO_format(text, predicted_output, common_words):
    bio_obj = BIO()
    df = bio_obj.convert_into_bio(text, predicted_output, common_words_list=common_words)
    return df

In [48]:
def get_dict_aspect(y,most_common_aspect):
    position=[]
    for innerlist in y:
        position.append([i for i, j in enumerate(innerlist) if j == 1])
    sorted_common=sorted(most_common_aspect)
    dict_aspect=[]
    for innerlist in position:
        inner_dict={}
        for word in sorted_common:
            if sorted_common.index(word) in innerlist:
                inner_dict[word]= 5
            else:
                inner_dict[word]=0
        dict_aspect.append(inner_dict)
    return dict_aspect

In [49]:

# Aspect term extractor of user's input.
#user_input=input("Enter a laptop review:\n\n")
user_input = "The battery life is really good and its size is reasonable"
#user_input = "it is of high quality, has a killer GUI, is extremely stable, is highly expandable, is bundled with lots of very good applications, is easy to use, and is absolutely gorgeous."
# Preprocessing and vectorizing
tagged_user_input = pos_tag([user_input])
#print(tagged_user_input)
filter_tagged_user_input = filter_tag(tagged_user_input)
#print(filter_tagged_user_input)

user_input_series = pd.Series(filter_tagged_user_input)
#print(user_input_series)
user_input_series_dtm = vect.transform(user_input_series)
#print(user_input_series_dtm)
# print(user_input_series[:5])

predict_aspect= lin_svc.predict(user_input_series_dtm)
#print(predict_aspect)
# predict_aspect_data = predict_aspect[0]
extra_feature=get_dict_aspect(predict_aspect, most_common_aspect)
extra_feature_dtm=DictVectorizer().fit_transform(extra_feature)
#extra_feature
df = BIO_format(user_input, predict_aspect[0], final_most_common_aspect)
df

,text,BIO
0,The,B
1,battery,B
2,life,I
3,is,O
4,really,O
5,good,O
6,and,O
7,its,O
8,size,B
9,is,O


In [50]:
df = BIO_format(user_input, predict_aspect[0], final_most_common_aspect)
df

,text,BIO
0,The,B
1,battery,B
2,life,I
3,is,O
4,really,O
5,good,O
6,and,O
7,its,O
8,size,B
9,is,O


In [51]:
# Aspect term extractor of user's input.
user_input=input("Enter a laptop review:\n\n")
#user_input = "The battery life is really good and its size is reasonable"
#user_input = "it is of high quality, has a killer GUI, is extremely stable, is highly expandable, is bundled with lots of very good applications, is easy to use, and is absolutely gorgeous."
# Preprocessing and vectorizing
tagged_user_input = pos_tag([user_input])
#print(tagged_user_input)
filter_tagged_user_input = filter_tag(tagged_user_input)
#print(filter_tagged_user_input)

user_input_series = pd.Series(filter_tagged_user_input)
#print(user_input_series)
user_input_series_dtm = vect.transform(user_input_series)
#print(user_input_series_dtm)
# print(user_input_series[:5])

predict_aspect= lin_svc.predict(user_input_series_dtm)
# predict_aspect_data = predict_aspect[0]
extra_feature=get_dict_aspect(predict_aspect, most_common_aspect)
extra_feature_dtm=DictVectorizer().fit_transform(extra_feature)
#extra_feature
df = BIO_format(user_input, predict_aspect[0], final_most_common_aspect)
df

Enter a laptop review:

This laptop is good for nothing


,text,BIO
0,This,O
1,laptop,O
2,is,O
3,good,O
4,for,O
5,nothing,O
